In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from keras.constraints import maxnorm
from keras.preprocessing.image import ImageDataGenerator

In [2]:
DATA_DIR = "/content/drive/MyDrive/malware_imagenes/malimg_paper_dataset_imgs"

In [3]:
!pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### PREPARACION

#### Carga de datos

In [4]:
path_images = ImageDataGenerator().flow_from_directory(directory=DATA_DIR, target_size=(128,128), batch_size=10000)

Found 9339 images belonging to 25 classes.


In [5]:
images, marks = next(path_images)

In [6]:
indices_path = path_images.class_indices

In [7]:
type_images = indices_path.keys()

In [8]:
add = sum(marks)
marks_qt = marks.shape[0]

In [9]:
quantity = ((add) / marks_qt)

In [10]:
quantity = quantity * 100
quantity

array([ 1.3063496 ,  1.2421031 , 31.577257  , 17.036085  ,  2.1201413 ,
        1.1350251 ,  1.5633365 ,  2.141557  ,  1.895278  ,  1.7346611 ,
        4.079666  ,  4.615055  ,  2.2807581 ,  1.9702322 ,  1.3170575 ,
        1.7025378 ,  1.4562588 ,  1.5205053 ,  1.6918299 ,  0.85662276,
        1.3705964 ,  1.4134276 ,  4.3687763 ,  1.038655  ,  8.566228  ],
      dtype=float32)

#### Preparacion de datos

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(images/255.,marks, test_size=0.3)

In [13]:
X_train.shape

(6537, 128, 128, 3)

In [14]:
X_test.shape

(2802, 128, 128, 3)

In [15]:
y_train.shape

(6537, 25)

In [16]:
y_test.shape

(2802, 25)

### MODELO BASE

In [17]:
def new_base_model():
  base_model = keras.Sequential()
  base_model.add(keras.layers.Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu', padding='same'))
  base_model.add(keras.layers.MaxPooling2D(2))
  base_model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
  base_model.add(keras.layers.Dense(64))
  base_model.add(keras.layers.Flatten())
  base_model.add(keras.layers.Dense(25, activation='softmax'))

  return base_model

In [65]:
base_model = new_base_model()
base_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 128, 128, 32)      2432      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 62, 62, 64)        18496     
                                                                 
 dense_6 (Dense)             (None, 62, 62, 64)        4160      
                                                                 
 flatten_3 (Flatten)         (None, 246016)            0         
                                                                 
 dense_7 (Dense)             (None, 25)                6150425   
                                                      

#### Compilado

In [25]:
base_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#### Clasificador

In [18]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

In [23]:
tf.compat.v1.disable_eager_execution()

In [26]:
classifier = KerasClassifier(
    model=base_model,
    clip_values=(0, 1),
    use_logits=False
)

#### Entrenamiento

In [27]:
classifier.fit(X_train, y_train, batch_size=64, nb_epochs=2)

Train on 6537 samples
Epoch 1/2
6537/6537 [==============================] - 15s 2ms/sample - loss: 1.1392 - accuracy: 0.7831
Epoch 2/2
6537/6537 [==============================] - 4s 617us/sample - loss: 0.0984 - accuracy: 0.9786


#### Predicciones

In [28]:
predictions = np.argmax(classifier.predict(X_test), axis=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [29]:
correct_predicted = np.sum(predictions == np.argmax(y_test, axis=1))
acc = correct_predicted / len(y_test)

print("Accuracy para predicciones de modelos base: {}%".format(acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(correct_predicted))

Accuracy para predicciones de modelos base: 97.39471805852962%
Cantidad de valores clasificados correctamente: 2729


#### Generacion de observaciones

In [50]:
attacker = FastGradientMethod(estimator=classifier, eps=50)

In [51]:
x_test_adv = attacker.generate(x=X_test)

#### Prediccion y resultados con observaciones falsas

In [52]:
adv_predictions = np.argmax(classifier.predict(x_test_adv), axis=1)

In [53]:
adv_correct_predicted = np.sum(adv_predictions == np.argmax(y_test, axis=1))
adv_acc = adv_correct_predicted / len(y_test)

print("Accuracy para modelos con ataque: {}%".format(adv_acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(adv_correct_predicted))

Accuracy para modelos con ataque: 1.2491077801570307%
Cantidad de valores clasificados correctamente: 35


Para mejorar nuestro modelo, y hacerlo más fuerte al momento de enfrentar estos ataques de evasión, existen dos opciones bastante viables, y que son facilitadas por la librería ART. Estas son, en esencia, la misma idea, pero se pueden ejecutar de manera distinta.

1. Entrenar el modelo con muestras adversarias generadas previamente. Con alguno de los ataques de evasion de la librería, podemos generar muestras adversarias, y agregarlas a nuestro dataset de entrenamiento. Después, entrenamos al modelo, y así al verse presentado con una imagen de ese tipo, podrá rendir de buena manera.

2. Utilizando la libreria de ART: AdversarialTrainer. Esta nos presenta un entrenamiento adversario original, que, al igual que lo que haríamos con la solución anterior, entrena al modelo, exponiéndolo a muestras generadas por atacantes adversarios. 

Esta segunda opción es la forma en la que mejoraremos el modelo, debido simplemente a la facilidad y abstracción en la implementación que la librería nos presenta. Además de la posibilidad de entrenar al modelo con varios tipos de posibles ataques.

### MODELO MEJORADO

#### Muestras adversarias

In [30]:
train_attacker = FastGradientMethod(estimator=classifier, eps=100)

In [31]:
x_train_fgm = attacker.generate(x=X_train)

In [32]:
X_train = np.append(X_train, x_train_fgm, axis=0)
y_train = np.append(y_train, y_train, axis=0)

#### Entrenamiento

In [ ]:
classifier.fit(X_train, y_train, batch_size=64, nb_epochs=3)

#### Predicciones y evaluacion

In [ ]:
rob_predictions = np.argmax(classifier.predict(x_test_adv), axis=1)

In [ ]:
rob_correct_predicted = np.sum(rob_predictions == np.argmax(y_test, axis=1))
rob_acc = rob_correct_predicted / len(y_test)

print("Accuracy de modelo mejorado con ataque: {}%".format(rob_acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(len(rob_correct_predicted)))

### Trainer de ART

#### Modelo

In [19]:
def new_defense_model():
  defense_model = keras.Sequential()
  defense_model.add(keras.layers.Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu', padding='same'))
  defense_model.add(keras.layers.MaxPooling2D(2))
  defense_model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
  defense_model.add(keras.layers.Dense(64))
  defense_model.add(keras.layers.Flatten())
  defense_model.add(keras.layers.Dense(25, activation='softmax'))

  return defense_model

In [68]:
defense_model = new_defense_model()
defense_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 128, 128, 32)      2432      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 62, 62, 64)        18496     
                                                                 
 dense_10 (Dense)            (None, 62, 62, 64)        4160      
                                                                 
 flatten_5 (Flatten)         (None, 246016)            0         
                                                                 
 dense_11 (Dense)            (None, 25)                6150425   
                                                      

In [32]:
defense_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#### Classifier

In [33]:
dclassifier = KerasClassifier(
    model=defense_model,
    clip_values=(0, 1),
    use_logits=False
)

#### Entrenamiento

In [34]:
dclassifier.fit(X_train, y_train, batch_size=64, nb_epochs=2)

Train on 6537 samples
Epoch 1/2
6537/6537 [==============================] - 4s 644us/sample - loss: 1.6294 - accuracy: 0.7545
Epoch 2/2
6537/6537 [==============================] - 4s 615us/sample - loss: 0.0891 - accuracy: 0.9761


#### Predicciones

In [35]:
predictions = np.argmax(dclassifier.predict(X_test), axis=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [36]:
correct_predicted = np.sum(predictions == np.argmax(y_test, axis=1))
acc = correct_predicted / len(y_test)

print("TOTAL DE VALORES: {}".format(len(y_test)))
print("Accuracy de modelo mejorado con ataque: {}%".format(acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(correct_predicted))

TOTAL DE VALORES: 2802
Accuracy de modelo mejorado con ataque: 96.03854389721627%
Cantidad de valores clasificados correctamente: 2691


#### Entrenamiento adversario

In [37]:
dattacker = FastGradientMethod(estimator=dclassifier, eps=100)

In [38]:
x_test_adv = dattacker.generate(x=X_test)

In [20]:
from art.defences.trainer import AdversarialTrainer

In [57]:
defender = AdversarialTrainer(
    classifier=dclassifier,
    attacks=dattacker,
    ratio=0.3
)

In [58]:
defender.fit(x=X_train, y=y_train, nb_epochs=5)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/5 [00:00<?, ?it/s]

In [59]:
av_predictions = np.argmax(dclassifier.predict(x_test_adv), axis=1)

In [60]:
av_correct_predicted = np.sum(av_predictions == np.argmax(y_test, axis=1))
av_acc = av_correct_predicted / len(y_test)

print("TOTAL DE VALORES: {}".format(len(y_test)))
print("Accuracy de modelo mejorado con ataque: {}%".format(av_acc * 100))
print("Cantidad de valores clasificados correctamente: {}".format(av_correct_predicted))

TOTAL DE VALORES: 2802
Accuracy de modelo mejorado con ataque: 80.69236259814419%
Cantidad de valores clasificados correctamente: 2261


#### Comparativa

In [42]:
N = len(y_test)

eps = [0.5, 1, 5, 10, 25]

classifier_acc = []
defender_acc = []


for e in eps:
  print("CALCULATING EPS {}".format(e))
  base_model = new_base_model()
  defense_model = new_defense_model()

  base_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
  )

  defense_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
  )  

  classifier = KerasClassifier(
    model=base_model,
    clip_values=(0, 1),
    use_logits=False
  )

  dclassifier = KerasClassifier(
    model=defense_model,
    clip_values=(0, 1),
    use_logits=False
  )

  classifier.fit(X_train, y_train, batch_size=64, nb_epochs=3)
  dclassifier.fit(X_train, y_train, batch_size=64, nb_epochs=3)

  attacker = FastGradientMethod(estimator=classifier, eps=e)
  dattacker = FastGradientMethod(estimator=dclassifier, eps=e)

  x_test_adv_1 = attacker.generate(x=X_test)
  x_test_adv_2 = dattacker.generate(x=X_test)

  defender = AdversarialTrainer(
    classifier=dclassifier,
    attacks=dattacker,
    ratio=0.3
  )

  defender.fit(x=X_train, y=y_train, nb_epochs=5)

  predictions_1 = np.sum(np.argmax(classifier.predict(x_test_adv_1), axis=1) == np.argmax(y_test, axis=1))
  predictions_2 = np.sum(np.argmax(defender.predict(x_test_adv_2), axis=1) == np.argmax(y_test, axis=1))

  classifier_acc.append(predictions_1)
  defender_acc.append(predictions_2)

CALCULATING EPS 0.5
Train on 6537 samples
Epoch 1/3
6537/6537 [==============================] - 4s 654us/sample - loss: 1.4772 - accuracy: 0.8037
Epoch 2/3
6537/6537 [==============================] - 4s 640us/sample - loss: 0.0920 - accuracy: 0.9801
Epoch 3/3
6537/6537 [==============================] - 4s 632us/sample - loss: 0.0171 - accuracy: 0.9979
Train on 6537 samples
Epoch 1/3
6537/6537 [==============================] - 4s 644us/sample - loss: 1.3942 - accuracy: 0.7687
Epoch 2/3
6537/6537 [==============================] - 4s 634us/sample - loss: 0.1034 - accuracy: 0.9694
Epoch 3/3
6537/6537 [==============================] - 4s 633us/sample - loss: 0.0218 - accuracy: 0.9960


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/5 [00:00<?, ?it/s]

CALCULATING EPS 1
Train on 6537 samples
Epoch 1/3
6537/6537 [==============================] - 4s 658us/sample - loss: 1.7298 - accuracy: 0.7494
Epoch 2/3
6537/6537 [==============================] - 4s 637us/sample - loss: 0.0948 - accuracy: 0.9723
Epoch 3/3
6537/6537 [==============================] - 4s 635us/sample - loss: 0.0267 - accuracy: 0.9931
Train on 6537 samples
Epoch 1/3
6537/6537 [==============================] - 4s 662us/sample - loss: 1.9480 - accuracy: 0.7523
Epoch 2/3
6537/6537 [==============================] - 4s 640us/sample - loss: 0.0874 - accuracy: 0.9801
Epoch 3/3
6537/6537 [==============================] - 4s 633us/sample - loss: 0.0279 - accuracy: 0.9954


...

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/5 [00:00<?, ?it/s]

CALCULATING EPS 25
Train on 6537 samples
Epoch 1/3
6537/6537 [==============================] - 4s 666us/sample - loss: 0.7764 - accuracy: 0.8177
Epoch 2/3
6537/6537 [==============================] - 4s 636us/sample - loss: 0.0482 - accuracy: 0.9890
Epoch 3/3
6537/6537 [==============================] - 4s 637us/sample - loss: 0.0103 - accuracy: 0.9986
Train on 6537 samples
Epoch 1/3
6537/6537 [==============================] - 4s 662us/sample - loss: 1.4775 - accuracy: 0.7742
Epoch 2/3
6537/6537 [==============================] - 4s 633us/sample - loss: 0.0925 - accuracy: 0.9735
Epoch 3/3
6537/6537 [==============================] - 4s 635us/sample - loss: 0.0414 - accuracy: 0.9913


Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/5 [00:00<?, ?it/s]

In [45]:
print("BASE MODEL")
for i in range(len(eps)): 
  print("* Accuracy for EPS" + " " * (4 - len(str(eps[i]))) + "{}: {:.2f}%".format(eps[i], (classifier_acc[i] / N) * 100))

print("\nDEFENSE MODEL")
for i in range(len(eps)): 
  print("* Accuracy for EPS" + " " * (4 - len(str(eps[i]))) + "{}: {:.2f}%".format(eps[i], (defender_acc[i] / N) * 100))

BASE MODEL
* Accuracy for EPS 0.5: 5.00%
* Accuracy for EPS   1: 5.42%
* Accuracy for EPS   5: 4.89%
* Accuracy for EPS  10: 5.39%
* Accuracy for EPS  25: 5.53%

DEFENSE MODEL
* Accuracy for EPS 0.5: 75.66%
* Accuracy for EPS   1: 59.53%
* Accuracy for EPS   5: 82.16%
* Accuracy for EPS  10: 86.94%
* Accuracy for EPS  25: 85.01%


Podemos observar que el modelo que ha pasado por el adversarial training es mucho más resiliente y se adapta mejor a las muestras falsas, de modo que logra clasificarlas correctamente en su mayoría.